In [12]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [13]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print((torch.cuda._get_nvml_device_index(0)))

True
NVIDIA A100-SXM4-80GB
3


In [14]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [15]:
# 임베딩 모델 초기화
embeddings = HuggingFaceEmbeddings(
    model_name="../ai_models/base_models/BGE-m3-ko",
    model_kwargs={'device': 'cuda:0'},
    encode_kwargs={'normalize_embeddings': True}
)

In [16]:
# def load_pdf_directory(directory_path):
#     loader = PyPDFDirectoryLoader(directory_path)
#     pages = loader.load()
#     return pages

def load_pdf_directory(directory_path):
    loader = PyPDFDirectoryLoader(directory_path)
    pages = loader.load()

    # 줄바꿈 노이즈 정리
    for page in pages:
        # 하이픈으로 줄바꿈된 단어 복원
        page.page_content = page.page_content.replace("-\n", "")
        # 일반 줄바꿈은 공백으로 변환
        page.page_content = page.page_content.replace("\n", " ")

    return pages

pdf_paths = "../data/pdf"
pdf_data = load_pdf_directory(pdf_paths)

In [17]:

def split_documents(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=50,
        length_function=len,
        separators=[r"\n{2,}", r"\n", r"[.!?]", r"[,;:]", r" "],
        is_separator_regex=True
    )
    return text_splitter.split_documents(documents)

chunks = split_documents(pdf_data)

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=400,  # 한 청크에 너무 많은 문장이 담기지 않도록
#     chunk_overlap=50,  # 앞뒤 문맥 연결 위해 소폭 겹침
#     separators=[r"\n{2,}", r"\n", r"[.!?]", r"[,;:]", r" "],  # 문단, 줄, 문장, 쉼표, 공백 순으로 분할
#     is_separator_regex=True
# )

# chunks = text_splitter.split_documents(pdf_data)

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=500,  # PDF 문서는 더 작은 청크로 나누는 것이 좋음
#     chunk_overlap=50, # 청크 간 중복도 줄임
#     length_function=len,
#     separators=["\n\n", "\n", ".", "!", "?", ";", ":", " ", ""],  # PDF 문서의 구조를 고려한 구분자 추가
#     is_separator_regex=False
# )

# chunks = text_splitter.split_documents(pdf_data)

In [18]:
# FAISS 벡터 스토어 생성
db = FAISS.from_documents(
    chunks, 
    embeddings
)

In [21]:
# FAISS 벡터 스토어 저장
faiss_index_directory = "./faiss_index_directory"
os.makedirs(faiss_index_directory, exist_ok=True)
db.save_local(faiss_index_directory)

In [19]:
# FAISS 벡터 스토어에서 유사도 검색
query = "피지컬 AI 란?"
docs = db.similarity_search(query, k=3)  # k는 반환할 문서 수

# 검색 결과 출력
for i, doc in enumerate(docs):
    print(f"\n## 검색결과 {i+1}")
    print(f"내용: {doc.page_content}")
    print(f"출처: {doc.metadata}")


## 검색결과 1
내용: ., 2021)의 연구에서 피지컬 AI는 디지털 AI의 확장 개념으로, “물리적 세계에서 자율적으로 작동하는 지능형 시스템”으로 정의∙ 저자들은 본 논문을 통해 피지컬  AI는 단순한  로봇  기술 을 넘 어, 센서·액추에 이터·AI 알고 리즘이  통합 된 시 스템으 로서 현실 환경에서의 상호작용성과 적응성을 핵심으로 하며, 차세대 인공지능의 핵심 패러다임이 될 수 있다고 강조●피지컬 AI 개념은 학계에서 기존의 체화된 AI(Embodied AI), 소프트 로보틱스(Soft Robotics), 사이버물리시스템(Cyber-Physical System, CPS), 적응형 AI(Adaptive AI) 등의 개념들과 밀접하게 연결∙ 이 중, 피지컬 AI와 가장 근접한 개념은 ‘체화된 AI’로
출처: {'producer': 'Hancom PDF 1.3.0.550', 'creator': 'Hwp 2018 10.0.0.14515', 'creationdate': '2025-05-13T13:38:44+09:00', 'author': 'Kevin', 'moddate': '2025-05-13T13:38:44+09:00', 'pdfversion': '1.4', 'source': '../data/pdf/IS-202 피지컬 AI의 현황과 시사점.pdf', 'total_pages': 48, 'page': 7, 'page_label': '8'}

## 검색결과 2
내용: SPRi 이슈리포트 IS-202 피지컬 AI의 현황과 시사점 5 (embodied reasoning)’ 능력을 부여하며, 이는 피지컬 AI가 현실 세계에서 자율성과 적응성을 갖춘 행동 실행 주체로 진화하는 방향성을 제시●미국의 AI 전문 인증기관 USAII(United States Artificial Intelligence Institute)*은 피지컬 AI를 “AI가 데이터 처리를 넘어 실제 세계에서 물리적 상호작용을 지능적으로 수행하는 기술”로 정의(USAII, 2025)* USAII는 글로

In [20]:
faiss_index_directory = "./faiss_index_directory"
vectorstore = FAISS.load_local(faiss_index_directory, embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(    
    search_type="similarity_score_threshold", 
    search_kwargs={"score_threshold": 0.5, "k": 3}
)

RuntimeError: Error in faiss::FileIOReader::FileIOReader(const char*) at /project/faiss/faiss/impl/io.cpp:67: Error: 'f' failed: could not open faiss_index_directory/index.faiss for reading: No such file or directory

In [11]:
vectorstore.similarity_search(k=3, query="ai 동향")

[Document(id='019c7d8e-5cce-4268-b0f2-09166c4e9ffa', metadata={'producer': 'Hancom PDF 1.3.0.550', 'creator': 'Hwp 2018 10.0.0.14515', 'creationdate': '2025-04-28T10:13:10+09:00', 'author': 'Sohyeon', 'moddate': '2025-04-28T10:13:10+09:00', 'pdfversion': '1.4', 'source': '../data/pdf/RE-189. 2024년 국내외 인공지능 산업 동향 연구.pdf', 'total_pages': 367, 'page': 357, 'page_label': '358'}, page_content='<표> 국내 AI 기업 동향'),
 Document(id='30d6ec03-d47c-49c1-aa59-e631c9a24ec2', metadata={'producer': 'Hancom PDF 1.3.0.550', 'creator': 'Hwp 2018 10.0.0.14515', 'creationdate': '2025-04-28T10:13:10+09:00', 'author': 'Sohyeon', 'moddate': '2025-04-28T10:13:10+09:00', 'pdfversion': '1.4', 'source': '../data/pdf/RE-189. 2024년 국내외 인공지능 산업 동향 연구.pdf', 'total_pages': 367, 'page': 359, 'page_label': '360'}, page_content='<표> 해외 AI 기업 동향'),
 Document(id='aee8e9e9-8c55-497b-af8f-25ef04bc223a', metadata={'producer': 'Hancom PDF 1.3.0.550', 'creator': 'Hwp 2018 10.0.0.14515', 'creationdate': '2025-04-28T10:13:10+09:00'